In [42]:
import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning,category=DeprecationWarning)
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from numpy.ma.core import mean
import math 
import matplotlib.pyplot as plt
import random


In [43]:
recipes = pd.read_csv("RAW_recipes.csv",encoding="Latin1")
ratings = pd.read_csv("RAW_interactions.csv")
ratings_v1=ratings.copy()
ratings_v1.drop(['rating'], axis=1, inplace=True)

In [44]:
unique_users=ratings_v1['user_id'].unique()

In [45]:
users_diseases = pd.DataFrame()
users_diseases['user_id']=pd.Series(unique_users)


In [46]:
users_diseases['Cancer']=False
users_diseases['Obesity']=False
users_diseases['Diabetes']=False
users_diseases['Cardiovascular']=False
users_diseases['Dental']=False
users_diseases['Osteoporosis']=False
users_diseases

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
0,38094,False,False,False,False,False,False
1,1293707,False,False,False,False,False,False
2,8937,False,False,False,False,False,False
3,126440,False,False,False,False,False,False
4,57222,False,False,False,False,False,False
...,...,...,...,...,...,...,...
226565,2000137415,False,False,False,False,False,False
226566,2001898209,False,False,False,False,False,False
226567,2002357020,False,False,False,False,False,False
226568,116593,False,False,False,False,False,False


In [47]:
healthy_users=users_diseases.sample(frac=0.4, random_state=1)
healthy_users.drop_duplicates(['user_id'], keep='last')
users_diseases=users_diseases[~ users_diseases['user_id'].isin(healthy_users['user_id'])]

In [48]:
users_diseases

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
0,38094,False,False,False,False,False,False
2,8937,False,False,False,False,False,False
5,52282,False,False,False,False,False,False
7,2000192946,False,False,False,False,False,False
8,76535,False,False,False,False,False,False
...,...,...,...,...,...,...,...
226564,2927134,False,False,False,False,False,False
226565,2000137415,False,False,False,False,False,False
226566,2001898209,False,False,False,False,False,False
226568,116593,False,False,False,False,False,False


In [49]:
cancers_gen1=users_diseases.sample(frac=0.08, random_state=1).drop_duplicates(['user_id'], keep='last')
cancers_gen1['Cancer']=True

cancers_gen2=cancers_gen1.sample(frac=0.312, random_state=1).drop_duplicates(['user_id'], keep='last')

cancers_gen3=cancers_gen2.sample(frac=0.2, random_state=1).drop_duplicates(['user_id'], keep='last')
cancers_gen3['Obesity']=True

cancers_gen4=cancers_gen2[~ cancers_gen2['user_id'].isin(cancers_gen3['user_id'])].drop_duplicates(['user_id'], keep='last')
cancers_gen4=cancers_gen4.sample(frac=0.15, random_state=1).drop_duplicates(['user_id'], keep='last')
cancers_gen4['Diabetes']=True

cancers_gen5=cancers_gen2[~ cancers_gen2['user_id'].isin(cancers_gen4['user_id'])& ~ cancers_gen2['user_id'].isin(cancers_gen3['user_id'])].drop_duplicates(['user_id'], keep='last')
cancers_gen5=cancers_gen5.sample(frac=0.2,  random_state=1).drop_duplicates(['user_id'], keep='last')
cancers_gen5['Cardiovascular']=True

cancers_gen6=cancers_gen2[~ cancers_gen2['user_id'].isin(cancers_gen4['user_id'])& ~ cancers_gen2['user_id'].isin(cancers_gen3['user_id'])& ~ cancers_gen2['user_id'].isin(cancers_gen5['user_id'])].drop_duplicates(['user_id'], keep='last')
cancers_gen6=cancers_gen6.sample(frac=0.3, random_state=1).drop_duplicates(['user_id'], keep='last')
cancers_gen6['Osteoporosis']=True

cancers_gen7=cancers_gen2[~ cancers_gen2['user_id'].isin(cancers_gen4['user_id'])& ~ cancers_gen2['user_id'].isin(cancers_gen3['user_id'])& ~ cancers_gen2['user_id'].isin(cancers_gen5['user_id'])& ~ cancers_gen2['user_id'].isin(cancers_gen6['user_id'])].drop_duplicates(['user_id'], keep='last')
cancers_gen7=cancers_gen7.sample(frac=0.15,  random_state=1).drop_duplicates(['user_id'], keep='last')
cancers_gen7['Dental']=True

In [50]:
cancer_with_other=pd.concat([cancers_gen3,cancers_gen4,cancers_gen5,cancers_gen6,cancers_gen7], axis=0).drop_duplicates()
tmp=cancers_gen1[~ cancers_gen1['user_id'].isin(cancer_with_other['user_id'])].drop_duplicates()
cancer_with_other

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
160131,473600,True,True,False,False,False,False
23031,2002082399,True,True,False,False,False,False
47054,1217341,True,True,False,False,False,False
7720,1803593023,True,True,False,False,False,False
104827,1939318,True,True,False,False,False,False
...,...,...,...,...,...,...,...
165095,1800195473,True,False,False,False,True,False
114794,2163258,True,False,False,False,True,False
128144,613309,True,False,False,False,True,False
169788,57769,True,False,False,False,True,False


In [51]:
cancer_with_2plus=tmp.sample(frac=0.245, random_state=1).drop_duplicates(['user_id'], keep='last')

cancer_with_2plus_1=cancer_with_2plus.sample(frac=0.208).drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_1['Obesity']=True
cancer_with_2plus_1['Cardiovascular']=True

cancer_with_2plus_2=cancer_with_2plus[~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_1['user_id'])].drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_2=cancer_with_2plus_2.sample(frac=0.088).drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_2['Dental']=True
cancer_with_2plus_2['Osteoporosis']=True

cancer_with_2plus_3=cancer_with_2plus[~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_1['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_2['user_id'])].drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_3=cancer_with_2plus_3.sample(frac=0.088).drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_3['Cardiovascular']=True
cancer_with_2plus_3['Diabetes']=True

cancer_with_2plus_4=cancer_with_2plus[~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_1['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_2['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_3['user_id'])].drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_4=cancer_with_2plus_4.sample(frac=0.088).drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_4['Diabetes']=True
cancer_with_2plus_4['Dental']=True

cancer_with_2plus_5=cancer_with_2plus[~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_1['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_2['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_3['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_4['user_id'])].drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_5=cancer_with_2plus_5.sample(frac=0.088).drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_5['Osteoporosis']=True
cancer_with_2plus_5['Diabetes']=True

cancer_with_2plus_6=cancer_with_2plus[~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_1['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_2['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_3['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_4['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_5['user_id'])].drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_6=cancer_with_2plus_6.sample(frac=0.088).drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_6['Osteoporosis']=True
cancer_with_2plus_6['Obesity']=True

cancer_with_2plus_7=cancer_with_2plus[~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_1['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_2['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_3['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_4['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_5['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_6['user_id'])].drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_7=cancer_with_2plus_7.sample(frac=0.088).drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_7['Osteoporosis']=True
cancer_with_2plus_7['Cardiovascular']=True

cancer_with_2plus_8=cancer_with_2plus[~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_1['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_2['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_3['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_4['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_5['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_6['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_7['user_id'])].drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_8=cancer_with_2plus_8.sample(frac=0.088).drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_8['Obesity']=True
cancer_with_2plus_8['Dental']=True

cancer_with_2plus_9=cancer_with_2plus[~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_1['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_2['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_3['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_4['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_5['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_6['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_7['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_8['user_id'])].drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_9=cancer_with_2plus_9.sample(frac=0.088).drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_9['Cardiovascular']=True
cancer_with_2plus_9['Dental']=True

cancer_with_2plus_10=cancer_with_2plus[~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_1['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_2['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_3['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_4['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_5['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_6['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_7['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_8['user_id'])& ~ cancer_with_2plus['user_id'].isin(cancer_with_2plus_9['user_id'])].drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_10=cancer_with_2plus_10.sample(frac=0.088).drop_duplicates(['user_id'], keep='last')
cancer_with_2plus_10['Obesity']=True
cancer_with_2plus_10['Diabetes']=True


In [52]:
cancer_with_2plus=pd.concat([cancer_with_2plus_1,cancer_with_2plus_2,cancer_with_2plus_3,cancer_with_2plus_4,cancer_with_2plus_5,cancer_with_2plus_6,cancer_with_2plus_7,cancer_with_2plus_8,cancer_with_2plus_9,cancer_with_2plus_10], axis=0).drop_duplicates()
cancer_with_2plus

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
143524,2071993,True,True,False,True,False,False
126952,160355,True,True,False,True,False,False
59318,2393570,True,True,False,True,False,False
133781,1508195,True,True,False,True,False,False
177795,2155498,True,True,False,True,False,False
...,...,...,...,...,...,...,...
163424,1580061,True,True,True,False,False,False
152681,2000133821,True,True,True,False,False,False
82548,350792,True,True,True,False,False,False
152767,835427,True,True,True,False,False,False


In [53]:
only_cancer=cancers_gen1[~ cancers_gen1['user_id'].isin(cancer_with_2plus['user_id'])& ~ cancers_gen1['user_id'].isin(cancer_with_other['user_id'])].drop_duplicates()
only_cancer


,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
108201,379153,True,False,False,False,False,False
190638,2000205405,True,False,False,False,False,False
202169,2125879,True,False,False,False,False,False
106001,2001594499,True,False,False,False,False,False
52085,305687,True,False,False,False,False,False
...,...,...,...,...,...,...,...
30907,387708,True,False,False,False,False,False
120133,2925580,True,False,False,False,False,False
122731,1577917,True,False,False,False,False,False
150067,1353236,True,False,False,False,False,False


In [54]:
cancer_dataset=pd.concat([only_cancer,cancer_with_other,cancer_with_2plus]).drop_duplicates()
cancer_dataset

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
108201,379153,True,False,False,False,False,False
190638,2000205405,True,False,False,False,False,False
202169,2125879,True,False,False,False,False,False
106001,2001594499,True,False,False,False,False,False
52085,305687,True,False,False,False,False,False
...,...,...,...,...,...,...,...
163424,1580061,True,True,True,False,False,False
152681,2000133821,True,True,True,False,False,False
82548,350792,True,True,True,False,False,False
152767,835427,True,True,True,False,False,False


In [55]:
diabetes_gen=users_diseases[~ users_diseases['user_id'].isin(cancer_dataset['user_id'])]
diabetes_gen1=diabetes_gen.sample(frac=0.16, random_state=1).drop_duplicates(['user_id'], keep='last')
diabetes_gen1['Diabetes']=True

diabetes_gen2=diabetes_gen1.sample(frac=0.293, random_state=1).drop_duplicates(['user_id'], keep='last')

diabetes_gen3=diabetes_gen2.sample(frac=0.5, random_state=1).drop_duplicates(['user_id'], keep='last')
diabetes_gen3['Obesity']=True

diabetes_gen4=diabetes_gen2[~ diabetes_gen2['user_id'].isin(diabetes_gen3['user_id'])].drop_duplicates(['user_id'], keep='last')
diabetes_gen4=diabetes_gen4.sample(frac=0.12, random_state=1).drop_duplicates(['user_id'], keep='last')
diabetes_gen4['Dental']=True

diabetes_gen5=diabetes_gen2[~ diabetes_gen2['user_id'].isin(diabetes_gen4['user_id'])& ~ diabetes_gen2['user_id'].isin(diabetes_gen3['user_id'])].drop_duplicates(['user_id'], keep='last')
diabetes_gen5=diabetes_gen5.sample(frac=0.2,  random_state=1).drop_duplicates(['user_id'], keep='last')
diabetes_gen5['Cardiovascular']=True

diabetes_gen6=diabetes_gen2[~ diabetes_gen2['user_id'].isin(diabetes_gen4['user_id'])& ~ diabetes_gen2['user_id'].isin(diabetes_gen3['user_id'])& ~ diabetes_gen2['user_id'].isin(diabetes_gen5['user_id'])].drop_duplicates(['user_id'], keep='last')
diabetes_gen6=diabetes_gen6.sample(frac=0.18, random_state=1).drop_duplicates(['user_id'], keep='last')
diabetes_gen6['Osteoporosis']=True

In [56]:
diabetes_with_other=pd.concat([diabetes_gen3,diabetes_gen4,diabetes_gen5,diabetes_gen6], axis=0).drop_duplicates()
tmp=diabetes_gen1[~ diabetes_gen1['user_id'].isin(diabetes_with_other['user_id'])].drop_duplicates()
diabetes_with_other

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
123949,2194364,False,True,True,False,False,False
154636,2000539205,False,True,True,False,False,False
2,8937,False,True,True,False,False,False
225611,1731405,False,True,True,False,False,False
71300,2128830,False,True,True,False,False,False
...,...,...,...,...,...,...,...
81665,641776,False,False,True,False,False,True
175380,2847075,False,False,True,False,False,True
5931,704166,False,False,True,False,False,True
150536,2705183,False,False,True,False,False,True


In [57]:
diabetes_gen1

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
97177,550810,False,False,True,False,False,False
221152,2001069849,False,False,True,False,False,False
213900,826352,False,False,True,False,False,False
190992,1090064,False,False,True,False,False,False
92208,257890,False,False,True,False,False,False
...,...,...,...,...,...,...,...
22280,1803533361,False,False,True,False,False,False
80863,2000385950,False,False,True,False,False,False
13289,450933,False,False,True,False,False,False
115363,1351181,False,False,True,False,False,False


In [58]:
diabetes_with_2plus=tmp.sample(frac=0.235, random_state=1).drop_duplicates(['user_id'], keep='last')

diabetes_with_2plus_1=diabetes_with_2plus.sample(frac=0.35).drop_duplicates(['user_id'], keep='last')
diabetes_with_2plus_1['Obesity']=True
diabetes_with_2plus_1['Cardiovascular']=True

diabetes_with_2plus_2=diabetes_with_2plus[~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_1['user_id'])].drop_duplicates(['user_id'], keep='last')
diabetes_with_2plus_2=diabetes_with_2plus_2.sample(frac=0.13).drop_duplicates(['user_id'], keep='last')
diabetes_with_2plus_2['Dental']=True
diabetes_with_2plus_2['Osteoporosis']=True


diabetes_with_2plus_3=diabetes_with_2plus[~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_1['user_id'])& ~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_2['user_id'])].drop_duplicates(['user_id'], keep='last')
diabetes_with_2plus_3=diabetes_with_2plus_3.sample(frac=0.13).drop_duplicates(['user_id'], keep='last')
diabetes_with_2plus_3['Osteoporosis']=True
diabetes_with_2plus_3['Obesity']=True

diabetes_with_2plus_4=diabetes_with_2plus[~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_1['user_id'])& ~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_2['user_id'])& ~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_3['user_id'])].drop_duplicates(['user_id'], keep='last')
diabetes_with_2plus_4=diabetes_with_2plus_4.sample(frac=0.13).drop_duplicates(['user_id'], keep='last')
diabetes_with_2plus_4['Osteoporosis']=True
diabetes_with_2plus_4['Cardiovascular']=True

diabetes_with_2plus_5=diabetes_with_2plus[~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_1['user_id'])& ~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_2['user_id'])& ~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_3['user_id'])& ~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_4['user_id'])].drop_duplicates(['user_id'], keep='last')
diabetes_with_2plus_5=diabetes_with_2plus_5.sample(frac=0.13).drop_duplicates(['user_id'], keep='last')
diabetes_with_2plus_5['Obesity']=True
diabetes_with_2plus_5['Dental']=True

diabetes_with_2plus_6=diabetes_with_2plus[~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_1['user_id'])& ~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_2['user_id'])& ~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_3['user_id'])& ~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_4['user_id'])& ~ diabetes_with_2plus['user_id'].isin(diabetes_with_2plus_5['user_id'])].drop_duplicates(['user_id'], keep='last')
diabetes_with_2plus_6=diabetes_with_2plus_6.sample(frac=0.13).drop_duplicates(['user_id'], keep='last')
diabetes_with_2plus_6['Cardiovascular']=True
diabetes_with_2plus_6['Dental']=True


In [59]:
diabetes_with_2plus=pd.concat([diabetes_with_2plus_1,diabetes_with_2plus_2,diabetes_with_2plus_3,diabetes_with_2plus_4,diabetes_with_2plus_5,diabetes_with_2plus_6], axis=0).drop_duplicates()
diabetes_with_2plus

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
1802,351811,False,True,True,True,False,False
139385,1097157,False,True,True,True,False,False
212674,1718496,False,True,True,True,False,False
41362,2537843,False,True,True,True,False,False
65559,2000905710,False,True,True,True,False,False
...,...,...,...,...,...,...,...
142990,751222,False,False,True,True,True,False
14484,421538,False,False,True,True,True,False
48643,434032,False,False,True,True,True,False
114673,2390775,False,False,True,True,True,False


In [60]:
only_diabetes=diabetes_gen1[~ diabetes_gen1['user_id'].isin(diabetes_with_2plus['user_id'])& ~ diabetes_gen1['user_id'].isin(diabetes_with_other['user_id'])].drop_duplicates()
only_diabetes


,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
97177,550810,False,False,True,False,False,False
221152,2001069849,False,False,True,False,False,False
213900,826352,False,False,True,False,False,False
190992,1090064,False,False,True,False,False,False
128099,584082,False,False,True,False,False,False
...,...,...,...,...,...,...,...
22280,1803533361,False,False,True,False,False,False
80863,2000385950,False,False,True,False,False,False
13289,450933,False,False,True,False,False,False
115363,1351181,False,False,True,False,False,False


In [61]:
diabetes_dataset=pd.concat([only_diabetes,diabetes_with_other,diabetes_with_2plus]).drop_duplicates()
diabetes_dataset

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
97177,550810,False,False,True,False,False,False
221152,2001069849,False,False,True,False,False,False
213900,826352,False,False,True,False,False,False
190992,1090064,False,False,True,False,False,False
128099,584082,False,False,True,False,False,False
...,...,...,...,...,...,...,...
142990,751222,False,False,True,True,True,False
14484,421538,False,False,True,True,True,False
48643,434032,False,False,True,True,True,False
114673,2390775,False,False,True,True,True,False


In [62]:
heart_gen=users_diseases[~ users_diseases['user_id'].isin(cancer_dataset['user_id'])& ~ users_diseases['user_id'].isin(diabetes_dataset['user_id'])]
heart_gen1=heart_gen.sample(frac=0.112, random_state=1).drop_duplicates(['user_id'], keep='last')
heart_gen1['Cardiovascular']=True

heart_gen2=heart_gen1.sample(frac=0.361, random_state=1).drop_duplicates(['user_id'], keep='last')

heart_gen3=heart_gen2.sample(frac=0.6, random_state=1).drop_duplicates(['user_id'], keep='last')
heart_gen3['Obesity']=True

heart_gen4=heart_gen2[~ heart_gen2['user_id'].isin(heart_gen3['user_id'])].drop_duplicates(['user_id'], keep='last')
heart_gen4=heart_gen4.sample(frac=0.1, random_state=1).drop_duplicates(['user_id'], keep='last')
heart_gen4['Dental']=True

heart_gen5=heart_gen2[~ heart_gen2['user_id'].isin(heart_gen4['user_id'])& ~ heart_gen2['user_id'].isin(heart_gen3['user_id'])].drop_duplicates(['user_id'], keep='last')
heart_gen5=heart_gen5.sample(frac=0.3,  random_state=1).drop_duplicates(['user_id'], keep='last')
heart_gen5['Osteoporosis']=True


In [63]:
heart_with_other=pd.concat([heart_gen3,heart_gen4,heart_gen5], axis=0).drop_duplicates()
tmp=heart_gen1[~ heart_gen1['user_id'].isin(heart_with_other['user_id'])].drop_duplicates()
heart_with_other

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
64347,2000887202,False,True,False,True,False,False
2056,53425,False,True,False,True,False,False
218488,2001397640,False,True,False,True,False,False
161832,771049,False,True,False,True,False,False
184946,735329,False,True,False,True,False,False
...,...,...,...,...,...,...,...
171730,808197,False,False,False,True,False,True
24849,631692,False,False,False,True,False,True
176292,2002033729,False,False,False,True,False,True
121641,312244,False,False,False,True,False,True


In [64]:
heart_with_2plus=tmp.sample(frac=0.363, random_state=1).drop_duplicates(['user_id'], keep='last')

heart_with_2plus_1=heart_with_2plus.sample(frac=0.15).drop_duplicates(['user_id'], keep='last')
heart_with_2plus_1['Obesity']=True
heart_with_2plus_1['Dental']=True

heart_with_2plus_2=heart_with_2plus[~ heart_with_2plus['user_id'].isin(heart_with_2plus_1['user_id'])].drop_duplicates(['user_id'], keep='last')
heart_with_2plus_2=heart_with_2plus_2.sample(frac=0.15).drop_duplicates(['user_id'], keep='last')
heart_with_2plus_2['Dental']=True
heart_with_2plus_2['Osteoporosis']=True


heart_with_2plus_3=heart_with_2plus[~ heart_with_2plus['user_id'].isin(heart_with_2plus_1['user_id'])& ~ heart_with_2plus['user_id'].isin(heart_with_2plus_2['user_id'])].drop_duplicates(['user_id'], keep='last')
heart_with_2plus_3=heart_with_2plus_3.sample(frac=0.7).drop_duplicates(['user_id'], keep='last')
heart_with_2plus_3['Osteoporosis']=True
heart_with_2plus_3['Obesity']=True





In [65]:
heart_with_2plus=pd.concat([heart_with_2plus_1,heart_with_2plus_2,heart_with_2plus_3], axis=0).drop_duplicates()
heart_with_2plus

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
79003,1759806,False,True,False,True,True,False
174111,4105,False,True,False,True,True,False
185397,2001947952,False,True,False,True,True,False
36905,681323,False,True,False,True,True,False
106579,2001121411,False,True,False,True,True,False
...,...,...,...,...,...,...,...
209737,2000142124,False,True,False,True,False,True
49083,2475814,False,True,False,True,False,True
224302,1244714,False,True,False,True,False,True
64175,625679,False,True,False,True,False,True


In [66]:
only_heart=heart_gen1[~ heart_gen1['user_id'].isin(heart_with_2plus['user_id'])& ~ heart_gen1['user_id'].isin(heart_with_other['user_id'])].drop_duplicates()
only_heart


,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
132569,1290185,False,False,False,True,False,False
161975,2001294690,False,False,False,True,False,False
20218,2001452752,False,False,False,True,False,False
134739,2818925,False,False,False,True,False,False
155030,2000921321,False,False,False,True,False,False
...,...,...,...,...,...,...,...
42772,574599,False,False,False,True,False,False
145539,1803807,False,False,False,True,False,False
187359,1213301,False,False,False,True,False,False
96557,721098,False,False,False,True,False,False


In [67]:
heart_dataset=pd.concat([only_heart,heart_with_other,heart_with_2plus]).drop_duplicates()
heart_dataset

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
132569,1290185,False,False,False,True,False,False
161975,2001294690,False,False,False,True,False,False
20218,2001452752,False,False,False,True,False,False
134739,2818925,False,False,False,True,False,False
155030,2000921321,False,False,False,True,False,False
...,...,...,...,...,...,...,...
209737,2000142124,False,True,False,True,False,True
49083,2475814,False,True,False,True,False,True
224302,1244714,False,True,False,True,False,True
64175,625679,False,True,False,True,False,True


In [68]:
ob_gen=users_diseases[~ users_diseases['user_id'].isin(cancer_dataset['user_id'])& ~ users_diseases['user_id'].isin(diabetes_dataset['user_id'])& ~ users_diseases['user_id'].isin(heart_dataset['user_id'])]
ob_gen1=ob_gen.sample(frac=0.208, random_state=1).drop_duplicates(['user_id'], keep='last')
ob_gen1['Obesity']=True

ob_gen2=ob_gen1.sample(frac=0.363, random_state=1).drop_duplicates(['user_id'], keep='last')

ob_gen3=ob_gen2.sample(frac=0.5, random_state=1).drop_duplicates(['user_id'], keep='last')
ob_gen3['Osteoporosis']=True

ob_gen4=ob_gen2[~ ob_gen2['user_id'].isin(ob_gen3['user_id'])].drop_duplicates(['user_id'], keep='last')
ob_gen4['Dental']=True


In [69]:
ob_with_other=pd.concat([ob_gen3,ob_gen4], axis=0).drop_duplicates()
tmp=ob_gen1[~ ob_gen1['user_id'].isin(ob_with_other['user_id'])].drop_duplicates()
ob_with_other

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
205095,158282,False,True,False,False,False,True
150803,261963,False,True,False,False,False,True
225224,2001444811,False,True,False,False,False,True
169533,2000905419,False,True,False,False,False,True
88563,647660,False,True,False,False,False,True
...,...,...,...,...,...,...,...
197472,66182,False,True,False,False,True,False
167289,2210526,False,True,False,False,True,False
120885,226893,False,True,False,False,True,False
97915,1414450,False,True,False,False,True,False


In [70]:
ob_with_2plus=tmp.sample(frac=0.361, random_state=1).drop_duplicates(['user_id'], keep='last')

ob_with_2plus['Osteoporosis']=True
ob_with_2plus['Dental']=True


In [71]:
ob_with_2plus

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
219925,2000057877,False,True,False,False,True,True
110166,2000381028,False,True,False,False,True,True
223795,1426254,False,True,False,False,True,True
194746,2805910,False,True,False,False,True,True
153478,2351857,False,True,False,False,True,True
...,...,...,...,...,...,...,...
126164,411986,False,True,False,False,True,True
180350,734206,False,True,False,False,True,True
106275,1310113,False,True,False,False,True,True
136370,2000858417,False,True,False,False,True,True


In [72]:
only_ob=ob_gen1[~ ob_gen1['user_id'].isin(ob_with_other['user_id'])& ~ ob_gen1['user_id'].isin(ob_with_2plus['user_id'])].drop_duplicates()
only_ob

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
161202,599136,False,True,False,False,False,False
132089,1179580,False,True,False,False,False,False
67154,2001460865,False,True,False,False,False,False
124373,22520,False,True,False,False,False,False
128735,152887,False,True,False,False,False,False
...,...,...,...,...,...,...,...
1413,2423451,False,True,False,False,False,False
112585,1589685,False,True,False,False,False,False
170331,228214,False,True,False,False,False,False
182397,2002242558,False,True,False,False,False,False


In [73]:
ob_dataset=pd.concat([only_ob,ob_with_other,ob_with_2plus]).drop_duplicates()
ob_dataset

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
161202,599136,False,True,False,False,False,False
132089,1179580,False,True,False,False,False,False
67154,2001460865,False,True,False,False,False,False
124373,22520,False,True,False,False,False,False
128735,152887,False,True,False,False,False,False
...,...,...,...,...,...,...,...
126164,411986,False,True,False,False,True,True
180350,734206,False,True,False,False,True,True
106275,1310113,False,True,False,False,True,True
136370,2000858417,False,True,False,False,True,True


In [74]:
den_gen=users_diseases[~ users_diseases['user_id'].isin(cancer_dataset['user_id'])& ~ users_diseases['user_id'].isin(diabetes_dataset['user_id'])& ~ users_diseases['user_id'].isin(heart_dataset['user_id'])& ~ users_diseases['user_id'].isin(ob_dataset['user_id'])]
den_gen1=den_gen.sample(frac=0.224, random_state=1).drop_duplicates(['user_id'], keep='last')
den_gen1['Dental']=True

den_with_other=den_gen1.sample(frac=0.333, random_state=1).drop_duplicates(['user_id'], keep='last')

den_with_other['Osteoporosis']=True



In [75]:
tmp=ob_gen1[~ ob_gen1['user_id'].isin(den_with_other['user_id'])].drop_duplicates()
den_with_other

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
134355,2001521403,False,False,False,False,True,True
66888,2873896,False,False,False,False,True,True
14989,1351262,False,False,False,False,True,True
100689,2000019511,False,False,False,False,True,True
95214,2002073514,False,False,False,False,True,True
...,...,...,...,...,...,...,...
2766,936601,False,False,False,False,True,True
4113,607820,False,False,False,False,True,True
96220,13141475,False,False,False,False,True,True
9782,2001726764,False,False,False,False,True,True


In [76]:
only_den=den_gen1[~ den_gen1['user_id'].isin(den_with_other['user_id'])].drop_duplicates()
only_den

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
48576,237946,False,False,False,False,True,False
97569,2625219,False,False,False,False,True,False
37140,459341,False,False,False,False,True,False
124923,2001151315,False,False,False,False,True,False
126142,2001714905,False,False,False,False,True,False
...,...,...,...,...,...,...,...
221465,867021,False,False,False,False,True,False
143479,322240,False,False,False,False,True,False
33191,546982,False,False,False,False,True,False
183752,2001578077,False,False,False,False,True,False


In [77]:
den_dataset=pd.concat([only_den,den_with_other]).drop_duplicates()
den_dataset

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
48576,237946,False,False,False,False,True,False
97569,2625219,False,False,False,False,True,False
37140,459341,False,False,False,False,True,False
124923,2001151315,False,False,False,False,True,False
126142,2001714905,False,False,False,False,True,False
...,...,...,...,...,...,...,...
2766,936601,False,False,False,False,True,True
4113,607820,False,False,False,False,True,True
96220,13141475,False,False,False,False,True,True
9782,2001726764,False,False,False,False,True,True


In [78]:
os_gen=users_diseases[~ users_diseases['user_id'].isin(cancer_dataset['user_id'])& ~ users_diseases['user_id'].isin(diabetes_dataset['user_id'])& ~ users_diseases['user_id'].isin(heart_dataset['user_id'])& ~ users_diseases['user_id'].isin(ob_dataset['user_id'])& ~ users_diseases['user_id'].isin(den_dataset['user_id'])]
os_gen['Osteoporosis']=True
os_dataset=os_gen
os_dataset

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
0,38094,False,False,False,False,False,True
7,2000192946,False,False,False,False,False,True
8,76535,False,False,False,False,False,True
16,68960,False,False,False,False,False,True
25,900992,False,False,False,False,False,True
...,...,...,...,...,...,...,...
226548,2000145340,False,False,False,False,False,True
226560,1197076,False,False,False,False,False,True
226564,2927134,False,False,False,False,False,True
226568,116593,False,False,False,False,False,True


In [79]:
dataset=pd.concat([cancer_dataset,diabetes_dataset,heart_dataset,ob_dataset,den_dataset,os_dataset,healthy_users]).drop_duplicates()
dataset

,user_id,Cancer,Obesity,Diabetes,Cardiovascular,Dental,Osteoporosis
108201,379153,True,False,False,False,False,False
190638,2000205405,True,False,False,False,False,False
202169,2125879,True,False,False,False,False,False
106001,2001594499,True,False,False,False,False,False
52085,305687,True,False,False,False,False,False
...,...,...,...,...,...,...,...
118125,2002076595,False,False,False,False,False,False
72219,1624985,False,False,False,False,False,False
102492,881063,False,False,False,False,False,False
147355,2184304,False,False,False,False,False,False


In [80]:
len(unique_users)

226570

In [81]:
dataset.to_csv("diseases_dataset.csv", 
                  index = None,
                  header=True)